In [6]:
import time

import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import solve
from scipy.sparse.linalg import cg

In [4]:
def chebyshev_solver(A, b, x0, max_iter=100, rho=None):
    # Step 1: Initializations
    x = x0.copy()
    r = b - A @ x

    # Estimate rho if not provided
    if rho is None:
        # Simple power method to estimate the spectral radius
        z = np.random.rand(len(x))
        for _ in range(10):
            z = A @ z
            rho = np.linalg.norm(z) / np.linalg.norm(z / np.linalg.norm(z))

    # Chebyshev coefficients
    d = (2 - rho**2) / 2
    omega = 1

    # Step 2: Iterations
    for k in range(max_iter):
        # Calculate the next iterate
        if k == 0:
            x_new = x + d * r
        else:
            omega_new = 4 / (4 - rho**2 * omega)
            x_new = x + omega_new * d * r + (1 - omega_new) * (x - x_prev)
            omega = omega_new

        # Update previous solution and residual
        x_prev = x
        x = x_new
        r = b - A @ x

        # Check for convergence
        if np.linalg.norm(r) < 1e-8:
            break

    return x

In [ ]:
def test_solvers(sizes):
    chebyshev_times = []
    scipy_direct_times = []
    scipy_cg_times = []

    for size in sizes:
        # Generate a symmetric positive definite matrix A and vector b
        A = np.random.rand(size, size)
        A = A + A.T + size * np.eye(size)  # Ensuring positive definiteness
        b = np.random.rand(size)
        x0 = np.zeros_like(b)

        # Time Chebyshev solver
        start_time = time.time()
        try:
            chebyshev_solver(A, b, x0, max_iter=100)
        except:
            pass
        chebyshev_time = time.time() - start_time
        chebyshev_times.append(chebyshev_time)

        # Time scipy direct solver
        start_time = time.time()
        solve(A, b)
        scipy_direct_time = time.time() - start_time
        scipy_direct_times.append(scipy_direct_time)

        # Time scipy CG solver
        start_time = time.time()
        cg(A, b, x0=x0, maxiter=100)
        scipy_cg_time = time.time() - start_time
        scipy_cg_times.append(scipy_cg_time)

    return chebyshev_times, scipy_direct_times, scipy_cg_times

# Test the solvers on matrix sizes ranging from 2 to 100
sizes = np.arange(2, 10001, 10)
chebyshev_times, scipy_direct_times, scipy_cg_times = test_solvers(sizes)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(sizes, chebyshev_times, label='Chebyshev Solver')
plt.plot(sizes, scipy_direct_times, label='Scipy Direct Solver')
plt.plot(sizes, scipy_cg_times, label='Scipy CG Solver')
plt.xlabel('Matrix Size')
plt.ylabel('Time (seconds)')
plt.title('Solver Performance Comparison')
plt.legend()
plt.grid(True)
plt.show()

<ipython-input-4-d0043a41075b>:25: RuntimeWarning: overflow encountered in multiply
  x_new = x + omega_new * d * r + (1 - omega_new) * (x - x_prev)
<ipython-input-4-d0043a41075b>:25: RuntimeWarning: invalid value encountered in add
  x_new = x + omega_new * d * r + (1 - omega_new) * (x - x_prev)
<ipython-input-4-d0043a41075b>:31: RuntimeWarning: overflow encountered in matmul
  r = b - A @ x
<ipython-input-4-d0043a41075b>:25: RuntimeWarning: invalid value encountered in subtract
  x_new = x + omega_new * d * r + (1 - omega_new) * (x - x_prev)
